# Time Consumption

In [ ]:
# Importing all needed libraries and functions

import numpy as np
import time
from tqdm import tqdm

# Plotting
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode()

# Custom functions
from generic_functions import *
from multivariate_images_tools import *
from change_detection_functions import *
from monte_carlo_tools import *

## Simulation parameters

In [ ]:
# General parameters
p = 10    # Dimension of vectors
N = 25    # Number of samples
T = 2     # Number of dates

# Enable parallel processing (or not)
enable_multi = False
# These two variables serves to split the original image into sub-images to be treated in parallel
# In general the optimal parameters are obtained for 
# number_of_threads = number of cores on the machine
number_of_threads = 4


# Number of Monte Carlo trials to compute histograms
number_of_trials = 4000  
number_of_bins_histogram = 30

# List of statistics to compute (take one for measure)
statistic_list = [covariance_equality_glrt_gaussian_statistic, 
                 covariance_equality_t1_gaussian_statistic,
                  covariance_equality_Wald_gaussian_statistic,
                 complex_Hotelling_Lawley_trace_statistic,
                  scale_and_shape_equality_robust_statistic,
                  Kullback_Leibler_statistic,
                  Similarity_measure_Meng_Liu,
                  Natural_Geodesic_statistic,
                Natural_student_t_distance,
                Wasserstein_Gaussian_statistic,
                 Wasserstein_robust_statistic]
statistic_names = [r'Gaussian GLRT', 
                   r't1 statistic',
                   r'Wald statistic',
                   r'HTL statistic',
                   r'MT statistic',
                  r'Kullback Leibler statistic',
                   r'MLL statistic',
                   r'Natural Gaussian Geodesic statistic',
                    r'Natural_student_t_distance',
                   r'Wasserstein Gaussian statistic',
                  r'Wasserstein robust statistic']
args_list = ['log', 'linear', 'log', None, (0.01, 15, 'log'), None, (0.01,15), None, (3,0.01,15),  None, (0.01,15)]
number_of_statistics = len(statistic_list)

## Measuring time

In [ ]:
# Parameters for Multivariate Complex Gaussian distribution
𝛍 = np.zeros(p)
pseudo_𝚺 = 0
ρ = 0.5  # List of Toeplitz matrices
τ= 10

# Setting useful variables for the Monte-Carlo
data_generation_function = generate_time_series_multivariate_vector
generation_function = wrapper_multivariate_complex_normal_samples
function_to_compute = compute_several_statistics
function_args = [statistic_list, args_list]

In [ ]:
# Doing the Monte Carlo simulation


# Generating Covariance matrix with given ρ
𝚺 = τ*ToeplitzMatrix(ρ, p)

# Generating parameters to pass to the Monte-Carlo function: since
# the series is homogeneous, we repeat T times
data_args_list = [[𝛍, 𝚺, N, pseudo_𝚺]]*T 
data_generation_args = [p, N, T, generation_function, data_args_list]

# Computing the Monte-Carlo simulation for this covariance matrix
t_0 = time.time()
λ = np.array(compute_monte_carlo_parallel(data_generation_function, data_generation_args, 
                                function_to_compute, function_args, 
                                number_of_trials, multi=enable_multi, number_of_threads=number_of_threads,
                                progressbar=not enable_multi))
t_1 = time.time()
print('Time consumption (%f)' % ((t_1-t_0)/number_of_trials))